In [190]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd

import time
import os
import csv
import json
import re
MOSTPOPULAR = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

In [191]:
def getFilmInfor(url):
    # instantiate a chrome options object so you can set the size and headless preference 
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument('blink-settings=imagesEnabled=false')  
    
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads 
    # and put it in the current directory 
    chrome_driver = os.getcwd() + "\\chromedriver2_win32" + "\\chromedriver.exe" 

    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
    driver.get(url)
    # get all id and title
    filmInfor = list()
    for film in driver.find_elements_by_xpath("//tbody[@class='lister-list']/tr"):
        title = film.find_element_by_xpath("td[2]/a").text
        filmId = film.find_element_by_xpath("td[4]/div").get_attribute("data-titleid")
        try:
            avgStars = film.find_element_by_xpath("td[3]").text
        except:
            avgStars = ''
        try:
           year = film.find_element_by_xpath("td[2]/span").text 
           year = re.findall(r"\d+",year)[0] #get the year
        except:
            year = ''
        filmInfor.append([filmId,title,year,avgStars])
    print(len(filmIdAndTitle))
    driver.quit()
    return filmInfor

In [192]:
def getReviews(filmId):
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--window-size=1920x1080") 
    chrome_options.add_argument('blink-settings=imagesEnabled=false') 
    url = "https://www.imdb.com/title/"+filmId+"/reviews"

    chrome_driver = os.getcwd() + "\\chromedriver2_win32" + "\\chromedriver.exe" 
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
    wait = WebDriverWait(driver,10)

    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    while True:
        try:
            driver.find_element_by_css_selector("button#load-more-trigger").click()
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".ipl-load-more__load-indicator")))
            soup = BeautifulSoup(driver.page_source, 'lxml')
        except Exception:break
 
    reviews = list()#[stars,text]
    for review in soup.find_all(class_='imdb-user-review'):
        if review.find("span",class_="spoiler-warning")==None:
            try:
                text = review.find(class_='show-more__control').get_text(strip=True)
                stars = review.find(class_="rating-other-user-rating").get_text(strip=True).split("/")[0]
                reviews.append([stars,text])
            except:
                pass
    driver.quit()
    return reviews

In [187]:
%%time
csvFile = open("most_popular_films.csv", 'wt', newline='\n', encoding='utf-8')
writer = csv.writer(csvFile)
header = ['id','name','year','avg_stars','stars','review']
writer.writerow(header)
for filmInfor in getFilmInfor(MOSTPOPULAR):
    temp = list()
    for review in getReviews(filmInfor[0]):
        temp.append(filmInfor+review)
    writer.writerows(temp)
    print(filmInfor[1]+"is done")
    print("-------------------------")
csvFile.close()

100
Wall time: 1min 36s


In [188]:
df = pd.read_csv("most_popular_films.csv")

In [189]:
df

,id,name,year,avg_stars,stars,review
0,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,10,Borat Make a *Glorious* #2! Subsequent Moviefi...
1,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,10,My husband loved SCB in all his incarnations (...
2,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,10,"What's even funnier than the movie itself, is ..."
3,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,10,For those saying Giuliani was just tucking in ...
4,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,10,Nothing more needs to be said than: LOOK at th...
...,...,...,...,...,...,...
774,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,5,The original Borat struck the world at the PER...
775,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,5,Doesn't have the same charm as the first one. ...
776,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,5,"A few laugh out loud moments, compared to the ..."
777,tt13143964,Borat Subsequent Moviefilm: Delivery of Prodig...,2020,6.9,6,"Jagshemash! In this movie, Borat goes back to ..."
